In [1]:
from langchain import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import requests
import os
import gradio as gr

# Set OpenAI and TMDb API keys
openai_api_key = "sk-svcacct-gCfZzvxqywYyhlW707rzn55aw_uE8OZgn-OJoWoZw0pii-LdD6dJN5gp4fBncT3BlbkFJBvpw97Xu267gUY7oWxNesfVzeTl2xjl9Hxj71UUf_Y7Y6fE_RMZ6v576KK67AA"  
tmdb_api_key = "7606a082a3cb4ebef8db46609a0994e0"  

# Define a function to query the TMDb API for movie recommendations
def query_tmdb(genre=None, min_year=None, max_year=None, min_rating=None, max_rating=None, max_runtime=None):
    base_url = "https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": tmdb_api_key,
        "language": "en-US",
        "sort_by": "popularity.desc",
        "include_adult": "false",
        "with_genres": genre,
        "primary_release_date.gte": f"{min_year}-01-01" if min_year else None,
        "primary_release_date.lte": f"{max_year}-12-31" if max_year else None,
        "vote_average.gte": min_rating,
        "vote_average.lte": max_rating,
        "with_runtime.lte": max_runtime,
        "page": 1
    }
    # Filter out None values in params
    params = {k: v for k, v in params.items() if v is not None}
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        movies = response.json().get("results", [])
        movie_details = []
        for movie in movies[:10]:  # Limit to top 10 results for performance
            # Get additional details, including runtime, by movie ID
            movie_id = movie['id']
            detail_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}", params={"api_key": tmdb_api_key})
            if detail_response.status_code == 200:
                details = detail_response.json()
                movie_details.append(
                    f"{movie['title']} ({movie['release_date'][:4]}), Rating: {movie.get('vote_average', 'N/A')}, "
                    f"Runtime: {details.get('runtime', 'N/A')} minutes"
                )
            else:
                movie_details.append(f"{movie['title']} - Additional details unavailable.")
        return movie_details
    else:
        return ["No results found."]

In [2]:
# Set up LangChain with prompt and memory
prompt_template = PromptTemplate(
    input_variables=["input", "history"],
    template="""You are a helpful assistant for movie recommendations. When users ask for movies, ask for genre, year, rating, or runtime if needed and then provide recommendations based on TMDb API.

    Conversation History:
    {history}

    User: {input}
    Assistant:"""
)
memory = ConversationBufferMemory(return_messages=True)
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
chatgpt_chain = ConversationChain(llm=llm, memory=memory, prompt=prompt_template)


C:\Users\scyph\AppData\Local\Temp\ipykernel_37276\3033465025.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
C:\Users\scyph\AppData\Local\Temp\ipykernel_37276\3033465025.py:13: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
C:\Users\scyph\AppData\Local\Temp\ipykernel_37276\3033465025.py:14: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_re

In [3]:
# Function to handle user queries and call the TMDb API based on input details
def chatgpt_direct_movie_recommender(user_query):
    # Get response from LangChain conversation chain
    chat_response = chatgpt_chain.predict(input=user_query)
    
    # Basic logic to detect if enough information is provided to query TMDb
    genre = 28 if "action" in user_query.lower() else None
    min_year, max_year = (2020, 2023) if "recent" in user_query.lower() else (None, None)
    
    # Detect rating range in user query, setting a default if none specified
    min_rating = 7.0 if "top rated" in user_query.lower() else None
    max_rating = 9.0 if "highly rated" in user_query.lower() else None
    
    # Query TMDb if sufficient parameters are detected
    if genre or min_year or min_rating:
        tmdb_results = query_tmdb(genre=genre, min_year=min_year, max_year=max_year, min_rating=min_rating, max_rating=max_rating)
        return "\n".join(tmdb_results)
    return chat_response

# Set up Gradio interface for interactive use
chatgpt_interface = gr.Interface(
    fn=chatgpt_direct_movie_recommender,
    inputs="text",
    outputs="text",
    title="Movie Recommender Bot"
)

# Launch the Gradio app
chatgpt_interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
